# Tone slicing

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [5]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [8]:
FONTSIZE = 20

In [9]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

# Functions

In [14]:
def generate_pairs(lst):
    """
    Generates all unique pairs from a list.

    Parameters:
    - lst (list): The list to generate pairs from.

    Returns:
    - list: A list of tuples, each containing a unique pair from the input list.
    """
    n = len(lst)
    return [(lst[i], lst[j]) for i in range(n) for j in range(i+1, n)]

## Inputs & Data

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [16]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000

In [17]:
BAND_TO_FREQ = {"theta": (6,11), "beta": (20,31), "gamma": (30,51)}

In [18]:
LFP_TRACES_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_competition_extention/results/2024_01_30_rce_2nd_cohort/proc/rce2_spectral_granger.pkl")

In [19]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_mPFC_granger_all_frequencies_all_windows,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[[1.216, 0.633, 0.282, 0.613, 0.636, 1.501, 0....","[[0.39, 0.2312, 0.02298, 0.1543, 0.315, 0.7856...","[[1.783, 1.615, 1.699, 0.9365, 1.126, 1.648, 0...","[[1.663, 0.849, 1.167, 0.758, 0.328, 0.7144, 1...","[[2.055, 0.452, 0.3164, 0.5933, 0.8477, 0.6006...","[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[[2.848, 1.116, 0.682, 0.198, 0.154, 0.127, 0....","[[1.97, 1.198, 0.10645, 0.2512, 0.3628, 0.595,...","[[0.09326, 0.08734, 0.136, 0.0357, 0.1394, 0.2...","[[1.11, 0.2117, 0.3718, 0.1334, 0.174, 0.7544,...","[[1.734, 1.35, 0.8564, 0.2327, 0.08453, 0.0556...","[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.01371, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[0.326, 0.1089, 0.624, 0.2566, 0.06903, 0.112...","[[0.4377, 0.1589, 0.2129, 0.3435, 0.5723, 1.25...","[[0.01956, 0.05756, 0.04272, 0.108, 0.4011, 1....","[[0.1444, 0.5464, 0.3662, 0.3176, 0.2191, 0.08...","[[0.8516, 0.1353, 0.897, 0.2006, 0.02, 0.1517,...","[[0.273, 0.1354, 0.1842, 0.1063, 0.9165, 0.894...","[[0.1587, 0.256, 0.2253, 0.1144, 0.04657, 0.24...","[[0.386, 0.0818, 0.07935, 0.03128, 0.2976, 1.4...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[0.05106, 0.03592, 0.804, 1.848, 0.3523, 1.55...","[[0.5166, nan, 0.236, 0.238, 0.282, 1.07, 0.03...","[[2.38, nan, 0.556, 0.719, 0.4656, 1.49, 1.627...","[[0.828, 0.4062, 0.2362, 0.0009437, 0.02545, 0...","[[0.

## Getting the slice of each tone

In [21]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'current_subject', 'subject',
       'all_subjects', 'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_

In [25]:
power_columns = [col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col]

In [26]:
power_columns

['mPFC_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'vHPC_power_all_frequencies_all_windows']

In [38]:
for col in power_columns:
    brain_region = col.split("_")[0]
    LFP_TRACES_DF["{}_power_all_frequencies_all_tones".format(brain_region)] = LFP_TRACES_DF.apply(lambda x: [x[col][np.logical_and((x["power_timestamps"] >= start), (x["power_timestamps"] <= end))] for (start, end) in x["tone_timestamps"]], axis=1)

In [43]:
coherence_columns = [col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col]

In [45]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows']

In [56]:
for col in coherence_columns:
    brain_region = "_".join(col.split("_")[:2])
    print(brain_region)
    LFP_TRACES_DF["{}_coherence_all_frequencies_all_tones".format(brain_region)] = LFP_TRACES_DF.apply(lambda x: [x[col][np.logical_and((x["coherence_timestamps"] >= start), (x["coherence_timestamps"] <= end))] for (start, end) in x["tone_timestamps"]], axis=1)

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


In [57]:
granger_columns = [col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col]

In [58]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows']

In [59]:
for col in granger_columns:
    brain_region = "_".join(col.split("_")[:2])
    print(brain_region)
    LFP_TRACES_DF["{}_granger_all_frequencies_all_tones".format(brain_region)] = LFP_TRACES_DF.apply(lambda x: [x[col][np.logical_and((x["granger_timestamps"] >= start), (x["granger_timestamps"] <= end))] for (start, end) in x["tone_timestamps"]], axis=1)

BLA_LH
LH_BLA
BLA_MD
MD_BLA
BLA_mPFC
mPFC_BLA
BLA_vHPC
vHPC_BLA
LH_MD
MD_LH
LH_mPFC
mPFC_LH
LH_vHPC
vHPC_LH
MD_mPFC
mPFC_MD
MD_vHPC
vHPC_MD
mPFC_vHPC
vHPC_mPFC


In [60]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,LH_mPFC_granger_all_frequencies_all_tones,mPFC_LH_granger_all_frequencies_all_tones,LH_vHPC_granger_all_frequencies_all_tones,vHPC_LH_granger_all_frequencies_all_tones,MD_mPFC_granger_all_frequencies_all_tones,mPFC_MD_granger_all_frequencies_all_tones,MD_vHPC_granger_all_frequencies_all_tones,vHPC_MD_granger_all_frequencies_all_tones,mPFC_vHPC_granger_all_frequencies_all_tones,vHPC_mPFC_granger_all_frequencies_all_tones
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[[[0.02559, 0.217, 0.2864, 0.1173, 0.013275, 0...","[[[0.7275, 0.395, 1.2, 0.419, 0.06555, 0.2083,...","[[[0.3564, 0.006203, 0.04623, 0.0738, 0.02225,...","[[[0.02269, 0.1727, 0.1677, 0.05524, 0.1885, 0...","[[[0.000793, 0.00719, 0.0329, 0.10266, 0.1116,...","[[[0.1024, 0.5317, 1.691, 0.333, 0.3748, 0.160...","[[[0.473, 0.12024, 0.3696, 0.1598, 0.08344, 0....","[[[1.354, 2.066, 0.2783, 0.543, 0.04102, 0.064...","[[[0.2996, 0.4377, 0.9536, 0.2041, 0.1239, 0.3...","[[[nan, 0.6577, 1.055, 0.669, 0.5767, 0.2238, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[[[0.892, 0.5625, 0.0394, 0.00834, 0.1284, 0.0...","[[[3.06, 0.5107, 0.783, 0.755, 0.6074, 0.4492,...","[[[0.0933, 0.0499, 0.0913, 0.3296, 0.2773, 0.2...","[[[0.0952, 0.693, 0.617, 0.656, 0.01146, 0.193...","[[[0.7627, 0.4146, 0.1236, 0.003307, 0.0941, 8...","[[[2.906, 0.1997, 0.653, 0.3142, 0.3665, 0.467...","[[[0.2329, 0.2856, 0.1501, 0.5786, 0.3108, 0.3...","[[[0.00647, 0.7847, 0.2278, 0.6094, 0.09735, 0...","[[[1.928, 0.3743, 0.2155, 0.2147, 0.58, 0.4675...","[[[nan, 0.715, 0.04117, 0.02257, 0.3499, 0.215..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[[nan, 0.0532, 0.536, 0.02722, 0.2935, 0.0189...","[[[0.1527, 0.1676, 0.7847, 0.0815, 0.2507, 0.1...","[[[0.05975, 0.08344, 0.6553, 0.4275, 1.307, 0....","[[[0.528, 0.1727, 0.05893, 0.05194, 0.0725, 0....","[[[nan, 0.0742, 0.512, 0.0797, 0.3408, 0.1895,...","[[[0.01932, 0.1401, 0.5957, 0.07996, 0.11487, ...","[[[0.10864, 0.5127, 0.7993, 0.913, 1.044, 0.74...","[[[0.791, 0.737, 0.02872, 0.1674, 0.1808, 0.08...","[[[1.057, 0.1718, 0.5137, 0.1847, 0.6343, 0.03...","[[[0.001826, 0.05145, 0.002533, 0.01627, 0.067..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[[0.706, 0.11206, 0.5894, 0.411, 0.924, 1.325...","[[[0.5166, 0.6353, 0.417, 0.5444, 0.1853, 0.40...","[[[0.06274, 0.0958, 0.2297, 0.672, 0.1718, 0.5...","[[[0.5063, 0.253, 0.0963, 0.1469, 0.023

In [74]:
COL_TO_KEEP = ['cohort', 'session_dir', 'original_file',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'current_subject', 'subject',
       'all_subjects', 'first_timestamp', 'last_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps']

,cohort,session_dir,original_file,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,all_subjects,first_timestamp,last_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,"[1.3, 1.4]",8798886,77093151,"[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,"[1.3, 1.4]",8798886,77093151,"[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,"[1.1, 1.2]",7977066,76318450,"[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,"[1.1, 1.2]",7977066,76318450,"[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [..."
4,2,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,1.1,"[1.1, 1.4]",7454593,75876357,"[[49358, 67558], [70155, 79355], [360955, 3671...","[[33624333, 33822933], [34132932, 34190535], [..."
5,2,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.4,1.4,"[1.1, 1.4]",7454593,75876357,"[[49358, 67558], [70155, 79355], [360955, 3671...","[[33624333, 33822933], [34132932, 34190535], [..."
6,2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...",20230614_114041_standard_comp_to_training_D3_s...,/scratch/back_up/reward_competition_extention/...,20230614_114041_standard_comp_to_training_D3_s...,1.1,1.1,"[1.1, 1.2]",34373723,102449020,"[[22157, 27562], [111755, 127157], [278955, 32...","[[33048338, 33059738], [33066538, 33067341], [..."
7,2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...",20230614_114041_standard_comp_to_training_D3_s...,

In [64]:
tone_columns = [col for col in LFP_TRACES_DF.columns if "tone" in col]

In [65]:
EXPLODED_LFP_TRACES_DF = pd.concat([LFP_TRACES_DF[col].explode() for col in tone_columns], axis=1)


In [77]:
MERGED_LFP_TRACES_DF = pd.merge(EXPLODED_LFP_TRACES_DF, LFP_TRACES_DF[COL_TO_KEEP], how="inner", left_index=True, right_index=True)

In [78]:
MERGED_LFP_TRACES_DF.columns

Index(['tone_frames', 'tone_timestamps',
       'mPFC_power_all_frequencies_all_tones',
       'MD_power_all_frequencies_all_tones',
       'LH_power_all_frequencies_all_tones',
       'BLA_power_all_frequencies_all_tones',
       'vHPC_power_all_frequencies_all_tones',
       'BLA_LH_coherence_all_frequencies_all_tones',
       'BLA_MD_coherence_all_frequencies_all_tones',
       'BLA_mPFC_coherence_all_frequencies_all_tones',
       'BLA_vHPC_coherence_all_frequencies_all_tones',
       'LH_MD_coherence_all_frequencies_all_tones',
       'LH_mPFC_coherence_all_frequencies_all_tones',
       'LH_vHPC_coherence_all_frequencies_all_tones',
       'MD_mPFC_coherence_all_frequencies_all_tones',
       'MD_vHPC_coherence_all_frequencies_all_tones',
       'mPFC_vHPC_coherence_all_frequencies_all_tones',
       'BLA_LH_granger_all_frequencies_all_tones',
       'LH_BLA_granger_all_frequencies_all_tones',
       'BLA_MD_granger_all_frequencies_all_tones',
       'MD_BLA_granger_all_frequenci

In [79]:
MERGED_LFP_TRACES_DF.head()

,tone_frames,tone_timestamps,mPFC_power_all_frequencies_all_tones,MD_power_all_frequencies_all_tones,LH_power_all_frequencies_all_tones,BLA_power_all_frequencies_all_tones,vHPC_power_all_frequencies_all_tones,BLA_LH_coherence_all_frequencies_all_tones,BLA_MD_coherence_all_frequencies_all_tones,BLA_mPFC_coherence_all_frequencies_all_tones,...,video_name,session_path,recording,current_subject,subject,all_subjects,first_timestamp,last_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,"[980, 1180]","[982229, 1182226]","[[0.03735, 0.02225, 0.02402, 0.0362, 0.03998, ...","[[0.4868, 0.5845, 0.594, 0.1986, 0.04138, 0.01...","[[0.013916, 0.01822, 0.01921, 0.0194, 0.01383,...","[[0.008415, 0.0122, 0.01417, 0.01495, 0.01073,...","[[0.001052, 0.01031, 0.02203, 0.02713, 0.02509...","[[0.3523, 0.1758, 0.1431, 0.2976, 0.7993, 0.85...","[[0.9194, 0.851, 0.5703, 0.682, 0.5513, 0.298,...","[[0.779, 0.7515, 0.06134, 0.2617, 0.7563, 0.85...",...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,"[1.3, 1.4]",8798886,77093151,"[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,"[3376, 3575]","[3382227, 3582224]","[[0.3538, 0.3096, 0.3787, 0.2727, 0.1991, 0.17...","[[0.03137, 0.01913, 0.04147, 0.04736, 0.05664,...","[[0.010414, 0.008224, 0.02959, 0.03525, 0.0552...","[[0.01521, 0.01127, 0.01417, 0.01839, 0.08905,...","[[1.356, 1.254, 1.23, 1.016, 0.709, 0.621, 0.5...","[[0.9272, 0.656, 0.5654, 0.5977, 0.7363, 0.758...","[[0.971, 0.946, 0.6934, 0.6626, 0.624, 0.5405,...","[[0.712, 0.9507, 0.4402, 0.2666, 0.3286, 0.535...",...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,"[1.3, 1.4]",8798886,77093151,"[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,"[5672, 5871]","[5682225, 5882222]","[[0.005627, 0.0082, 0.02856, 0.02502, 0.02675,...","[[0.002995, 0.006123, 0.01249, 0.01134, 0.0106...","[[0.00457, 0.01611, 0.02945, 0.02557, 0.01985,...","[[0.001195, 0.006264, 0.01048, 0.013275, 0.015...","[[0.007504, 0.01108, 0.01573, 0.01001, 0.00817...","[[0.8438, 0.9785, 0.9688, 0.8286, 0.7593, 0.73...","[[0.4626, 0.8745, 0.8203, 0.6245, 0.5596, 0.59...","[[0.92, 0.921, 0.899, 0.74, 0.7314, 0.7783, 0....",...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,"[1.3, 1.4]",8798886,77093151,"[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,"[7468, 7668]","[7482224, 7682221]","[[0.005745, 0.02194, 0.05325, 0.04877, 0.03702...","[[0.003006, 0.00328, 0.004528, 0.006573, 0.002...","[[0.003529, 0.005146, 0.00888, 0.01143, 0.0079...","[[0.004852, 0.00552, 0.006077, 0.00874, 0.0052...","[[0.02538, 0.02655, 0.04022, 0.03918, 0.02568,...","[[0.902, 0.953, 0.815, 0.8394, 0.8403, 0.899, ...","[[0.648, 0.9155, 0.515, 0.7007, 0.6255, 0.8145...","[[0.000729, 0.266, 0.3923, 0.4963, 0.6743, 0.6...",...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,"[1.3, 1.4]",8798886,77093151,"[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
0,"[8566, 8765]","[8582220, 8782223]","[[0.02032, 0.01335, 0.06146, 0.0738, 0.06052, ...","[[0.007442, 0.0293, 0.0334, 0.04352, 0.02269, ...","[[0.00676, 0.02718, 0.03287, 0.0481, 0.03308, ...","[[0.000627, 0.0271, 0.04706, 0.05704, 0.0778, ...","[[0.0007944, 0.02773, 0.0326, 0.03049, 0.01246...","[[0.9985, 0.7417, 0.64, 0.7617, 0.6187, 0.0867...","[[0.9995, 0.706, 0.554, 0.708, 0.671, 0.11505,...","[[0.3945, 0.2184, 0.55, 0.7046, 0.674, 0.4905,...",...,20230612_101430_standard_comp_to_training_

In [81]:
MERGED_LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_tone_sliced.pkl")

In [20]:
raise ValueError()

ValueError: 

# Power Calcuation

- Getting the column name of all the traces

In [15]:
trace_columns = [col for col in LFP_TRACES_DF.columns if "lfp_trace" in col]

In [16]:
trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [17]:
for col in trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float16))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


- Calcuating the power at each frequency band

In [18]:
for col in trace_columns:
    brain_region = col.split("_")[0]
    print(brain_region)

    # Define column names
    lfp_trace_col = f"{brain_region}_lfp_trace"
    multitaper_col = f"{brain_region}_power_multitaper"
    connectivity_col = f"{brain_region}_power_connectivity"
    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        # Apply Multitaper function to the lfp_trace column
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF[lfp_trace_col].apply(
            lambda x: Multitaper(
                time_series=x, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT,
                time_window_duration=TIME_WINDOW_DURATION, 
                time_window_step=TIME_WINDOW_STEP
            )
        )

        # Apply Connectivity function to the multitaper column
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and power functions to the connectivity column
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.power().squeeze()
        )
        
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[power_col].apply(lambda x: x.astype(np.float16))
            
        # Removing unnecessary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
    
    except Exception as e: 
        print(e)

mPFC
MD
LH
BLA
vHPC


- Getting the timestamps of the power

In [19]:
LFP_TRACES_DF["power_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])
# .iloc[0][500:-500:500].shape

- Making sure that the timestamps for power makes sense with shape and values

In [20]:
LFP_TRACES_DF["power_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6833,)
3    (6833,)
4    (6840,)
Name: power_timestamps, dtype: object

In [21]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [22]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [23]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [24]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col]].head()

,mPFC_power_frequencies,MD_power_frequencies,LH_power_frequencies,BLA_power_frequencies,vHPC_power_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [25]:
LFP_TRACES_DF["power_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [26]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col], errors="ignore")

In [27]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,mPFC_power_all_frequencies_all_windows,MD_power_all_frequencies_all_windows,LH_power_all_frequencies_all_windows,BLA_power_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,power_timestamps,power_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[0.8945, 0.962, 1.193, 1.322, 1.038, 0.572, 0....","[0.6655, 0.661, 0.8735, 1.023, 0.7085, 0.2559,...","[0.2328, 0.3381, 0.5566, 0.7466, 0.8315, 0.822...","[[0.0417, 0.08026, 0.08984, 0.0713, 0.03882, 0...","[[0.00632, 0.01746, 0.02122, 0.02435, 0.0162, ...","[[0.01959, 0.04858, 0.0577, 0.0546, 0.0346, 0....","[[0.0376, 0.02425, 0.0459, 0.0485, 0.0531, 0.0...","[[0.009636, 0.00574, 0.01634, 0.01555, 0.01978...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[0.8115, 0.72, 0.623, 0.646, 0.652, 0.3887, 0....","[0.7983, 1.067, 1.111, 0.8345, 1.065, 1.497, 1...","[0.8823, 1.2295, 1.485, 1.232, 1.067, 1.286, 1...","[[0.0002556, 0.0005894, 0.002153, 0.002962, 0....","[[0.01222, 0.01273, 0.010864, 0.003828, 0.0028...","[[0.01642, 0.01765, 0.01605, 0.006023, 0.00426...","[[0.000475, 0.002134, 0.006367, 0.007313, 0.00...","[[0.001258, 0.001891, 0.005817, 0.00642, 0.006...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[0.003944, 0.2524, 0.489, 0.4082, 0.215, -0.04...","[0.001743, 0.0401, 0.129, 0.1935, 0.1063, -0.1...","[0.241, 0.3652, 0.3152, -0.0394, -0.2773, -0.2...","[[0.02177, 0.00988, 0.02951, 0.01837, 0.03044,...","[[0.009155, 0.00615, 0.005947, 0.002455, 0.003...","[[0.01685, 0.01133, 0.009186, 0.003273, 0.0048...","[[0.00824, 0.005802, 0.00613, 0.003384, 0.0024...","[[0.001064, 0.006977, 0.006905, 0.01109, 0.006...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[-0.3896, -0.8096, -0.4617, 0.3958, 0.04495, -...","[0.2257, -0.0932, 0.1667, 0.7603, 0.5005, 0.14...","[1.249, 1.02, 0.978, 1.081, 0.8228, 0.563, 0.4...","[[0.1814, 0.2057, 0.2612, 0.1082, 0.0678, 0.04...","[[0.2328, 0.2329, 0.301, 0.1137, 0.07825, 0.05...","[[0.3687, 0.3687, 0.4849, 0.1884, 0.1263, 0.06...","[[0.339, 0.2898, 0.3713, 0.1

## Coherence Calculation

- Getting the trace column pairs

In [28]:
trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [29]:
brain_region_pairs = generate_pairs(sorted(trace_columns))
brain_region_pairs = sorted(brain_region_pairs)


In [30]:
brain_region_pairs

[('BLA_lfp_trace', 'LH_lfp_trace'),
 ('BLA_lfp_trace', 'MD_lfp_trace'),
 ('BLA_lfp_trace', 'mPFC_lfp_trace'),
 ('BLA_lfp_trace', 'vHPC_lfp_trace'),
 ('LH_lfp_trace', 'MD_lfp_trace'),
 ('LH_lfp_trace', 'mPFC_lfp_trace'),
 ('LH_lfp_trace', 'vHPC_lfp_trace'),
 ('MD_lfp_trace', 'mPFC_lfp_trace'),
 ('MD_lfp_trace', 'vHPC_lfp_trace'),
 ('mPFC_lfp_trace', 'vHPC_lfp_trace')]

## Coherece Calculation

- Calculating the coherence

In [31]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_coherence_multitaper"
        connectivity_col = f"{pair_base_name}_coherence_connectivity"
        frequencies_col = f"{pair_base_name}_coherence_frequencies"
        coherence_col = f"{pair_base_name}_coherence_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )

        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and coherence functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.coherence_magnitude()[:,:,0,1]
        )

        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[coherence_col].apply(lambda x: x.astype(np.float16))

    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


- Getting the timestamps of the coherence

In [32]:
LFP_TRACES_DF["coherence_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for coherence makes sense with shape and values

In [33]:
LFP_TRACES_DF["coherence_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6833,)
3    (6833,)
4    (6840,)
Name: coherence_timestamps, dtype: object

In [34]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [35]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [36]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [37]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col]].head()

,BLA_LH_coherence_frequencies,BLA_MD_coherence_frequencies,BLA_mPFC_coherence_frequencies,BLA_vHPC_coherence_frequencies,LH_MD_coherence_frequencies,LH_mPFC_coherence_frequencies,LH_vHPC_coherence_frequencies,MD_mPFC_coherence_frequencies,MD_vHPC_coherence_frequencies,mPFC_vHPC_coherence_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [38]:
LFP_TRACES_DF["coherence_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [39]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col], errors="ignore")

In [40]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,BLA_mPFC_coherence_all_frequencies_all_windows,BLA_vHPC_coherence_all_frequencies_all_windows,LH_MD_coherence_all_frequencies_all_windows,LH_mPFC_coherence_all_frequencies_all_windows,LH_vHPC_coherence_all_frequencies_all_windows,MD_mPFC_coherence_all_frequencies_all_windows,MD_vHPC_coherence_all_frequencies_all_windows,mPFC_vHPC_coherence_all_frequencies_all_windows,coherence_timestamps,coherence_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[[0.888, 0.745, 0.7627, 0.573, 0.5938, 0.253, ...","[[0.8647, 0.9487, 0.8594, 0.887, 0.2456, 0.050...","[[0.8984, 0.9756, 0.98, 0.966, 0.9023, 0.9233,...","[[0.985, 0.9976, 0.995, 0.9863, 0.991, 0.7124,...","[[0.5127, 0.671, 0.6772, 0.729, 0.4365, 0.2107...","[[0.96, 0.9683, 0.9746, 0.949, 0.8516, 0.716, ...","[[0.8125, 0.6313, 0.6074, 0.619, 0.2313, 0.282...","[[0.6343, 0.714, 0.6826, 0.702, 0.477, 0.4604,...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[[0.372, 0.5474, 0.856, 0.553, 0.0804, 0.316, ...","[[0.993, 0.9385, 0.9673, 0.8193, 0.746, 0.736,...","[[0.997, 0.9976, 0.9917, 0.962, 0.9106, 0.816,...","[[0.01095, 0.2644, 0.2732, 0.2174, 0.141, 0.36...","[[0.08765, 0.8496, 0.1497, 0.4238, 0.4124, 0.2...","[[0.01811, 0.2402, 0.2756, 0.3364, 0.2107, 0.5...","[[0.05984, 0.8145, 0.1871, 0.592, 0.475, 0.332...","[[0.2944, 0.401, 0.797, 0.9, 0.3484, 0.509, 0....","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[0.9824, 0.8765, 0.6265, 0.3909, 0.7974, 0.92...","[[0.2443, 0.3582, 0.3428, 0.03363, 0.3813, 0.6...","[[0.9985, 0.9927, 0.948, 0.857, 0.932, 0.9956,...","[[0.9907, 0.8687, 0.5107, 0.12445, 0.587, 0.89...","[[0.385, 0.3591, 0.3196, 0.02284, 0.256, 0.628...","[[0.9966, 0.8594, 0.672, 0.2869, 0.5376, 0.893...","[[0.3481, 0.3923, 0.3418, 0.01117, 0.2186, 0.6...","[[0.299, 0.10486, 0.0316, 0.01532, 0.0652, 0.4...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[0.8716, 0.9595, 0.9404, 0.851, 0.8643, 0.980...","[[0.941, 0.99, 0.971, 0.9087, 0.875, 0.451, 0....","[[0.9873, 0.995, 0.9937, 0.9907, 0.9575, 0.968...","[[0.9194, 0.984, 0.963, 0.921, 0.9043,

In [41]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_coherence.pkl")

# Calculate Granger's

In [42]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    region_1_base_name = region_1.split('_')[0]
    region_2_base_name = region_2.split('_')[0]

    pair_base_name = f"{region_1_base_name}_{region_2_base_name}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_granger_multitaper"
        connectivity_col = f"{pair_base_name}_granger_connectivity"
        frequencies_col = f"{pair_base_name}_granger_frequencies"
        granger_1_2_col = f"{region_1_base_name}_{region_2_base_name}_granger_all_frequencies_all_windows"
        granger_2_1_col = f"{region_2_base_name}_{region_1_base_name}_granger_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )
    
        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and granger functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        
        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,0,1]
        )

        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,1,0]
        )

        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[granger_1_2_col].apply(lambda x: x.astype(np.float16))
        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[granger_2_1_col].apply(lambda x: x.astype(np.float16))
        
    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

BLA_LH


Maximum iterations reached. 6753 of 6828 converged
Maximum iterations reached. 6814 of 6828 converged
Maximum iterations reached. 6812 of 6833 converged
Maximum iterations reached. 6831 of 6833 converged
Maximum iterations reached. 6814 of 6840 converged
Maximum iterations reached. 6837 of 6841 converged
Maximum iterations reached. 6788 of 6804 converged
Maximum iterations reached. 6376 of 6678 converged
Maximum iterations reached. 6725 of 6746 converged
Maximum iterations reached. 6743 of 6746 converged
Maximum iterations reached. 7939 of 7968 converged
Maximum iterations reached. 7959 of 7968 converged
Maximum iterations reached. 6241 of 6559 converged
Maximum iterations reached. 6709 of 6858 converged
Maximum iterations reached. 3898 of 4117 converged
Maximum iterations reached. 6787 of 6835 converged
Maximum iterations reached. 6816 of 6835 converged
Maximum iterations reached. 6978 of 6979 converged
Maximum iterations reached. 6720 of 7034 converged
Maximum iterations reached. 718

BLA_MD


Maximum iterations reached. 6779 of 6828 converged
Maximum iterations reached. 6819 of 6828 converged
Maximum iterations reached. 6810 of 6833 converged
Maximum iterations reached. 6828 of 6833 converged
Maximum iterations reached. 6805 of 6840 converged
Maximum iterations reached. 6836 of 6841 converged
Maximum iterations reached. 6786 of 6804 converged
Maximum iterations reached. 6381 of 6678 converged
Maximum iterations reached. 6734 of 6746 converged
Maximum iterations reached. 6742 of 6746 converged
Maximum iterations reached. 7928 of 7968 converged
Maximum iterations reached. 7947 of 7968 converged
Maximum iterations reached. 6238 of 6559 converged
Maximum iterations reached. 6709 of 6858 converged
Maximum iterations reached. 3898 of 4117 converged
Maximum iterations reached. 6780 of 6835 converged
Maximum iterations reached. 6819 of 6835 converged
Maximum iterations reached. 6976 of 6979 converged
Maximum iterations reached. 6714 of 7034 converged
Maximum iterations reached. 718

BLA_mPFC


...

Maximum iterations reached. 6689 of 6828 converged
Maximum iterations reached. 6827 of 6828 converged
Maximum iterations reached. 6824 of 6833 converged
Maximum iterations reached. 6828 of 6833 converged
Maximum iterations reached. 6829 of 6840 converged
Maximum iterations reached. 6797 of 6804 converged
Maximum iterations reached. 6366 of 6678 converged
Maximum iterations reached. 6734 of 6746 converged
Maximum iterations reached. 7957 of 7968 converged
Maximum iterations reached. 7955 of 7968 converged
Maximum iterations reached. 6253 of 6559 converged
Maximum iterations reached. 6707 of 6858 converged
Maximum iterations reached. 3897 of 4117 converged
Maximum iterations reached. 6818 of 6835 converged
Maximum iterations reached. 6826 of 6835 converged
Maximum iterations reached. 6729 of 7034 converged
Maximum iterations reached. 7187 of 7204 converged
Maximum iterations reached. 7149 of 7175 converged
Maximum iterations reached. 7174 of 7175 converged
Maximum iterations reached. 722

MD_vHPC


Maximum iterations reached. 6751 of 6828 converged
Maximum iterations reached. 6820 of 6828 converged
Maximum iterations reached. 6832 of 6833 converged
Maximum iterations reached. 6840 of 6841 converged
Maximum iterations reached. 6389 of 6678 converged
Maximum iterations reached. 6744 of 6746 converged
Maximum iterations reached. 6743 of 6746 converged
Maximum iterations reached. 6274 of 6559 converged
Maximum iterations reached. 6661 of 6858 converged
Maximum iterations reached. 3899 of 4117 converged
Maximum iterations reached. 6834 of 6835 converged
Maximum iterations reached. 6832 of 6835 converged
Maximum iterations reached. 6732 of 7034 converged
Maximum iterations reached. 7200 of 7204 converged
Maximum iterations reached. 7166 of 7175 converged
Maximum iterations reached. 7170 of 7175 converged
Maximum iterations reached. 7234 of 7241 converged
Maximum iterations reached. 6830 of 6831 converged
Maximum iterations reached. 6828 of 6831 converged
Maximum iterations reached. 683

mPFC_vHPC


Maximum iterations reached. 6723 of 6828 converged
Maximum iterations reached. 6831 of 6833 converged
Maximum iterations reached. 6839 of 6840 converged
Maximum iterations reached. 6389 of 6678 converged
Maximum iterations reached. 6745 of 6746 converged
Maximum iterations reached. 7967 of 7968 converged
Maximum iterations reached. 7965 of 7968 converged
Maximum iterations reached. 6274 of 6559 converged
Maximum iterations reached. 6691 of 6858 converged
Maximum iterations reached. 3897 of 4117 converged
Maximum iterations reached. 6833 of 6835 converged
Maximum iterations reached. 6732 of 7034 converged
Maximum iterations reached. 7200 of 7204 converged
Maximum iterations reached. 7163 of 7175 converged
Maximum iterations reached. 7174 of 7175 converged
Maximum iterations reached. 7239 of 7241 converged
Maximum iterations reached. 6826 of 6831 converged
Maximum iterations reached. 6839 of 6842 converged
Maximum iterations reached. 6840 of 6842 converged
Maximum iterations reached. 685

- Getting the timestamps of the granger

In [43]:
LFP_TRACES_DF["granger_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for granger makes sense with shape and values

In [44]:
LFP_TRACES_DF["granger_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6833,)
3    (6833,)
4    (6840,)
Name: granger_timestamps, dtype: object

In [45]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [46]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [47]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [48]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col]].head()

,BLA_LH_granger_frequencies,BLA_MD_granger_frequencies,BLA_mPFC_granger_frequencies,BLA_vHPC_granger_frequencies,LH_MD_granger_frequencies,LH_mPFC_granger_frequencies,LH_vHPC_granger_frequencies,MD_mPFC_granger_frequencies,MD_vHPC_granger_frequencies,mPFC_vHPC_granger_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [49]:
LFP_TRACES_DF["granger_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [50]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col], errors="ignore")

In [51]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_mPFC_granger_all_frequencies_all_windows,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[[1.216, 0.633, 0.282, 0.613, 0.636, 1.501, 0....","[[0.39, 0.2312, 0.02298, 0.1543, 0.315, 0.7856...","[[1.783, 1.615, 1.699, 0.9365, 1.126, 1.648, 0...","[[1.663, 0.849, 1.167, 0.758, 0.328, 0.7144, 1...","[[2.055, 0.452, 0.3164, 0.5933, 0.8477, 0.6006...","[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[[2.848, 1.116, 0.682, 0.198, 0.154, 0.127, 0....","[[1.97, 1.198, 0.10645, 0.2512, 0.3628, 0.595,...","[[0.09326, 0.08734, 0.136, 0.0357, 0.1394, 0.2...","[[1.11, 0.2117, 0.3718, 0.1334, 0.174, 0.7544,...","[[1.734, 1.35, 0.8564, 0.2327, 0.08453, 0.0556...","[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.01371, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[0.326, 0.1089, 0.624, 0.2566, 0.06903, 0.112...","[[0.4377, 0.1589, 0.2129, 0.3435, 0.5723, 1.25...","[[0.01956, 0.05756, 0.04272, 0.108, 0.4011, 1....","[[0.1444, 0.5464, 0.3662, 0.3176, 0.2191, 0.08...","[[0.8516, 0.1353, 0.897, 0.2006, 0.02, 0.1517,...","[[0.273, 0.1354, 0.1842, 0.1063, 0.9165, 0.894...","[[0.1587, 0.256, 0.2253, 0.1144, 0.04657, 0.24...","[[0.386, 0.0818, 0.07935, 0.03128, 0.2976, 1.4...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[0.05106, 0.03592, 0.804, 1.848, 0.3523, 1.55...","[[0.5166, nan, 0.236, 0.238, 0.282, 1.07, 0.03...","[[2.38, nan, 0.556, 0.719, 0.4656, 1.49, 1.627...","[[0.828, 0.4062, 0.2362, 0.0009437, 0.02545, 0...","[[0.

## Calculating the averages

In [56]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'current_subject', 'subject',
       'all_subjects', 'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_

In [52]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")

In [54]:
raise ValueError()

ValueError: 